#NOTE: To make it easier for us to manage datasets, images and models we create a HOME constant.

In [ ]:
# @title
import os
HOME = os.getcwd()
print("HOME:", HOME)

#Install necessary libraries

In [ ]:
# @title
!pip install torch torchvision torchaudio
!pip install tqdm
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
%cd {HOME}

import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

#Install the dependencies of grounding dino

In [ ]:
# @title
%cd {HOME}/GroundingDINO
!pip install -e .

#Download Grounding DINO Model Weights

To run Grounding DINO we need two files - configuration and model weights. The configuration file is part of the Grounding DINO repository, which we have already cloned. The weights file, on the other hand, we need to download. We write the paths to both files to the GROUNDING_DINO_CONFIG_PATH and GROUNDING_DINO_CHECKPOINT_PATH variables and verify if the paths are correct and the files exist on disk.

In [ ]:
# @title
import os

GROUNDING_DINO_CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(GROUNDING_DINO_CONFIG_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CONFIG_PATH))

In [ ]:
# @title
%cd {HOME}
!mkdir -p {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

In [ ]:
# @title
import os

GROUNDING_DINO_CHECKPOINT_PATH = os.path.join(HOME, "weights", "groundingdino_swint_ogc.pth")
print(GROUNDING_DINO_CHECKPOINT_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CHECKPOINT_PATH))

#Download Segment Anything Model (SAM) Weights
As with Grounding DINO, in order to run SAM we need a weights file, which we must first download. We write the path to local weight file to SAM_CHECKPOINT_PATH variable and verify if the path is correct and the file exist on disk.

In [ ]:
# @title
%cd {HOME}
!mkdir -p {HOME}/weights
%cd {HOME}/weights

!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [ ]:
# @title
import os

SAM_CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(SAM_CHECKPOINT_PATH, "; exist:", os.path.isfile(SAM_CHECKPOINT_PATH))

#Load Models

In [ ]:
# @title
# Change to the correct directory
%cd /content/GroundingDINO

import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from groundingdino.util.inference import Model
from segment_anything import sam_model_registry, SamPredictor

# Ensure no CUDA is being used
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
#os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

# Set the device to CPU
#DEVICE = torch.device('cpu')
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

grounding_dino_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH, device=DEVICE)

# Load the SAM model
SAM_ENCODER_VERSION = "vit_h"
sam = sam_model_registry[SAM_ENCODER_VERSION](checkpoint=SAM_CHECKPOINT_PATH).to(device=DEVICE)
sam_predictor = SamPredictor(sam)

print("Models loaded successfully.")

In [ ]:
# @title
!pip install --upgrade sympy

#Ensure GPU access

In [ ]:
# @title
print("Checking GPU availability...")
gpu_available = torch.cuda.is_available()
print("GPU available:", gpu_available)
if not gpu_available:
    print("Please enable GPU support for better performance.")

#Functions

In [ ]:
# @title
def enhance_class_name(class_names):
    return [f"all {class_name}s" for class_name in class_names]

def segment(sam_predictor, image, xyxy):
    sam_predictor.set_image(image)
    result_masks = []
    for box in xyxy:
        masks, scores, logits = sam_predictor.predict(box=box, multimask_output=True)
        index = np.argmax(scores)
        result_masks.append(masks[index])
    return np.array(result_masks)

def list_files_with_extensions(directory, extensions):
    file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(tuple(extensions)):
                file_paths.append(os.path.join(root, file))
    return file_paths

#Full Dataset Mask Auto Annotation

In [ ]:
# @title
def full_dataset_annotation(images_directory, classes, box_threshold, text_threshold):
    images = {}
    annotations = {}
    image_paths = list_files_with_extensions(directory=images_directory, extensions=['jpg', 'jpeg', 'png'])

    print(f"Found {len(image_paths)} image(s) in {images_directory}")

    for image_path in tqdm(image_paths):
        print(f"Processing image: {image_path}")
        image_name = os.path.basename(image_path)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Failed to read image: {image_path}")
            continue

        detections = grounding_dino_model.predict_with_classes(
            image=image,
            classes=enhance_class_name(class_names=classes),
            box_threshold=box_threshold,
            text_threshold=text_threshold
        )
        if detections is None or len(detections.xyxy) == 0:
            print(f"No detections for image: {image_path}")
            continue

        detections.mask = segment(sam_predictor=sam_predictor, image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB), xyxy=detections.xyxy)
        images[image_name] = image
        annotations[image_name] = detections
    print(f"Processed {len(images)} image(s)")
    return images, annotations

#Prompt the user for class names

In [ ]:
# @title
def get_class_names():
    while True:
        class_names_input = input("Please enter the class names, separated by commas (e.g., dog, cat, car): ")
        class_names = [name.strip() for name in class_names_input.split(",") if name.strip()]
        if class_names:
            return class_names
        else:
            print("You must enter at least one class name.")

class_names = get_class_names()
print(class_names)


# Ensure the images directory exists in Colab
# Upload or ensure images are present in '/content/folder_name'

In [ ]:
# @title
import os
print(os.getcwd())
images_directory = os.path.join(HOME, '/content/annotation_images/Cauliflower')
os.makedirs(images_directory, exist_ok=True)
print(class_names)
images, annotations = full_dataset_annotation(images_directory, class_names, 0.35, 0.25)

# Check if images and annotations are generated

In [ ]:
# @title
print(f"Number of images: {len(images)}")
print(f"Number of annotations: {len(annotations)}")

# Display 2-3 random annotated images

In [ ]:
# @title
import supervision as sv
import random
import cv2
import matplotlib.pyplot as plt

def display_random_annotated_images(images, annotations, class_names, num_images=3):
    plot_images = []
    plot_titles = []

    box_annotator = sv.BoxAnnotator()
    mask_annotator = sv.MaskAnnotator()

    selected_images = random.sample(list(annotations.keys()), min(num_images, len(annotations)))

    for image_name in selected_images:
        image = images[image_name]
        detections = annotations[image_name]

        # Annotate the image with masks
        annotated_image = mask_annotator.annotate(scene=image.copy(), detections=detections)

        # Annotate the image with bounding boxes
        annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)

        # Add labels to the annotated image using OpenCV
        for i in range(len(detections.class_id)):
            class_id = detections.class_id[i]
            confidence = detections.confidence[i]
            bbox = detections.xyxy[i]  # Assuming detections has an attribute xyxy

            label = f"{class_names[class_id]}: {confidence:.2f}"
            x1, y1, x2, y2 = map(int, bbox)

            # Draw the label above the bounding box
            cv2.putText(annotated_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        plot_images.append(annotated_image)
        title = " ".join(set([
            str(class_names[class_id])
            for class_id
            in detections.class_id
        ]))
        plot_titles.append(title)

    # Display the images in a grid
    fig, axs = plt.subplots(1, min(num_images, len(annotations)), figsize=(15, 5))
    for idx, img in enumerate(plot_images):
        axs[idx].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[idx].set_title(plot_titles[idx])
        axs[idx].axis('off')
    plt.show()

# Assuming images, annotations, and class_names are already defined
display_random_annotated_images(images, annotations, class_names, num_images=3)

#Prompt user to enter main folder name

In [ ]:
# @title
main_folder_name = input("Please enter the main folder name for the dataset: ")

# Create directories in YOLO format

In [ ]:
# @title

def create_yolo_directories(main_folder_name):
    base_path = os.path.join(HOME, main_folder_name)
    for split in ['train', 'test', 'valid']:
        os.makedirs(os.path.join(base_path, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(base_path, split, 'labels'), exist_ok=True)
    return base_path

base_path = create_yolo_directories(main_folder_name)

# Save labels in YOLO format

In [ ]:
# @title
def save_annotations(images, annotations, classes, base_path, split):
    images_path = os.path.join(base_path, split, 'images')
    labels_path = os.path.join(base_path, split, 'labels')

    for image_name, detections in annotations.items():
        # Save image
        image = images[image_name]
        image_output_path = os.path.join(images_path, image_name)
        cv2.imwrite(image_output_path, image)
        print(f"Saved image to {image_output_path}")

        # Save annotations
        annotation_output_path = os.path.join(labels_path, f"{os.path.splitext(image_name)[0]}.txt")
        with open(annotation_output_path, 'w') as f:
            for bbox, confidence, class_id, mask in zip(detections.xyxy, detections.confidence, detections.class_id, detections.mask):
                x_center = (bbox[0] + bbox[2]) / 2 / image.shape[1]
                y_center = (bbox[1] + bbox[3]) / 2 / image.shape[0]
                width = (bbox[2] - bbox[0]) / image.shape[1]
                height = (bbox[3] - bbox[1]) / image.shape[0]
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
        print(f"Saved annotations to {annotation_output_path}")

# Split data

In [ ]:
# @title
train_ratio = 0.8
valid_ratio = 0.1
test_ratio = 0.1

image_paths = list(images.keys())
np.random.shuffle(image_paths)
train_split = int(len(image_paths) * train_ratio)
valid_split = int(len(image_paths) * (train_ratio + valid_ratio))

train_images = {k: images[k] for k in image_paths[:train_split]}
valid_images = {k: images[k] for k in image_paths[train_split:valid_split]}
test_images = {k: images[k] for k in image_paths[valid_split:]}

train_annotations = {k: annotations[k] for k in image_paths[:train_split]}
valid_annotations = {k: annotations[k] for k in image_paths[train_split:valid_split]}
test_annotations = {k: annotations[k] for k in image_paths[valid_split:]}

print("Saving training annotations...")
save_annotations(train_images, train_annotations, class_names, base_path, 'train')
print("Saving validation annotations...")
save_annotations(valid_images, valid_annotations, class_names, base_path, 'valid')
print("Saving test annotations...")
save_annotations(test_images, test_annotations, class_names, base_path, 'test')

print(f"YOLO dataset has been created in {base_path}")

#Download files from collab in zip format

In [ ]:
# @title
import locale
import shutil
import os
from google.colab import files

def set_utf8_locale():
    """Sets the locale to UTF-8."""
    try:
        locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')  # Try en_US.UTF-8 first
    except locale.Error:
        try:
            locale.setlocale(locale.LC_ALL, 'UTF-8')  # Try UTF-8 as a fallback
        except locale.Error:
            print("Warning: Could not set locale to UTF-8. File names might be corrupted.")

# Call the function before running zip command
set_utf8_locale()

# Define the directory to be zipped and the output zip file path
input_directory = '/content/Annotated_Cauliflower'
output_zip_file = '/content/Cauliflower.zip'

# Create a ZIP file using shutil.make_archive
shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', input_directory)

# Download the created ZIP file
files.download(output_zip_file)

# Install the yolo packages

In [ ]:
# @title
import locale

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# Install necessary packages without locale checks
!pip install --no-deps ultralytics opencv-python-headless

# Verify installation by importing the packages
import ultralytics
import cv2

print("Packages installed successfully.")

# Import the libraries

In [ ]:
# @title
import os
import cv2
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
from google.colab import files

# Load the YOLO model (pre-trained or custom)

In [ ]:
# @title
def load_model(model_path=None):
    """
    Load YOLO model (either from a pretrained or custom path)
    """
    if model_path:
        model = YOLO(model_path)
    else:
        model = YOLO('yolov10s.pt')
    return model

# Function to train the YOLO model with custom dataset

In [ ]:
# @title
def train_model(yaml_path, model, epochs=25, batch=16, imgsz=640):
    """
    Train the YOLO model with the given parameters.
    """
    # Start training
    results = model.train(
        data=yaml_path,
        epochs=epochs,
        batch=batch,
        imgsz=imgsz,
    )

    # Get the path to the trained weights
    last_weights_path = results.save_dir / "weights" / "last.pt"  # Path to the last weights
    best_weights_path = results.save_dir / "weights" / "best.pt"  # Path to the best weights

    print(f"Training completed. Model is saved at:")
    print(f"Last weights: {last_weights_path}")
    print(f"Best weights: {best_weights_path}")

    return model

# Function for inference on a single image

In [ ]:
# @title
import cv2
import matplotlib.pyplot as plt

def detect_on_image(model, image_path):
    """
    Perform detection on a single image with improved label placement.
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return

    # Perform detection
    results = model(image)

    # Draw bounding boxes and labels
    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Box coordinates
        label = model.names[int(box.cls[0])]  # Class name
        confidence = box.conf[0]  # Confidence score

        # Prepare the label text
        label_text = f"{label} ({confidence:.2f})"

        # Get text size
        (text_width, text_height), baseline = cv2.getTextSize(
            label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1
        )

        # Adjust label position to stay within bounds
        label_y = max(y1 - text_height - baseline, 0)  # Stay within image bounds
        label_x = x1

        # Draw a filled rectangle behind the text
        cv2.rectangle(
            image,
            (label_x, label_y),
            (label_x + text_width, label_y + text_height + baseline),
            (0, 255, 0),  # Background color (green)
            -1,  # Filled rectangle
        )

        # Draw the bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Draw the label text
        cv2.putText(
            image,
            label_text,
            (label_x, label_y + text_height),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 0, 0),  # Text color (black for contrast)
            1,
        )

    # Convert BGR image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the result using matplotlib
    plt.imshow(image)
    plt.axis("off")
    plt.show()


# Function for real-time detection using the webcam

In [ ]:
# @title
def real_time_detection(model):
    """
    Perform real-time ingredient detection using the webcam.
    """
    cap = cv2.VideoCapture(0)  # Open default camera

    if not cap.isOpened():
        print("Error: Could not access the camera.")
        return

    print("Press 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        # Perform detection
        results = model(frame)

        # Draw bounding boxes and labels
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Box coordinates
            label = model.names[int(box.cls[0])]  # Class name
            confidence = box.conf[0]  # Confidence score

            # Draw the bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                frame,
                f"{label} ({confidence:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                2,
            )

        # Display the frame
        cv2.imshow("Ingredient Detection", frame)

        # Quit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

# Save the YOLO model for future use

In [ ]:
# @title
def save_model(model, output_path="/content/saved_model"):
    """
    Save the trained YOLO model for future use.
    """
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    model.save(os.path.join(output_path, "trained_model.pt"))
    print(f"Model saved to {output_path}")


In [ ]:
# @title
from google.colab import drive
drive.mount('/content/drive')

# Main function to train, detect and save the model

In [ ]:
# @title
def main(mode="image", yaml_path=None, image_path=None, save=False, output_path="/content/saved_model"):
    if not yaml_path:
        print("Error: Provide a valid YAML file path for training.")
        return

    # Load the model (either pre-trained or custom)
    model = load_model('/content/saved_model/trained_model.pt')
    # Uncomment the following line if you need to train the model
    model = train_model(yaml_path, model)

    # Perform detection based on the user's choice
    if mode == "image" and image_path:
        detect_on_image(model, image_path)
    elif mode == "realtime":
        real_time_detection(model)
    else:
        print("Error: Invalid mode. Choose 'image' or 'realtime'.")

    # Save the trained model if requested
    if save:
        save_model(model, output_path)

# Example usage
if __name__ == "__main__":
    # Example: Detect ingredients from an image or real-time
    # Main mode can be 'image' or 'realtime'

    # main(mode="image", yaml_path="path_to_your_dataset.yaml", image_path="path_to_image.jpg", save=True)
    # Or use real-time mode
    main(mode="image", yaml_path="/content/data.yaml", image_path="/content/train/images/0010_Cabbage_train.jpg", save=False)

In [ ]:
# @title
import torch
from ultralytics import YOLO

# Load the trained model
model = YOLO('/content/saved_model/trained_model.pt')

# Load an image
image_path = '/content/train/images/0020_Brinjal_train.jpg'
image = cv2.imread(image_path)

# Run inference
results = model(image)

# Display results for each detection
for result in results:
    result.show()

In [ ]:
# @title
import shutil
import os

# Specify the directory you want to delete
dir_path = '/content/train'

# Check if the directory exists
if os.path.exists(dir_path) and os.path.isdir(dir_path):
    # Delete the directory
    shutil.rmtree(dir_path)
    print(f"Directory '{dir_path}' has been deleted.")
else:
    print(f"Directory '{dir_path}' does not exist.")

# Convert model to tflite format

In [ ]:
# @title
# Install YOLOv8 and related tools
!pip install ultralytics onnx onnx-simplifier tensorflow tf2onnx onnx-tf

#Upload Your YOLOv8 .pt Model

In [ ]:
# @title
from google.colab import files
uploaded = files.upload()  # Upload your `.pt` file

#Export the YOLOv8 Model to ONNX

In [ ]:
# @title
from ultralytics import YOLO

# Load your YOLOv8 model
model = YOLO('trained_model.pt')  # Replace with your uploaded .pt filename

# Export the model to ONNX
model.export(format='onnx', opset=12)  # Adjust opset if needed

#Simplify the ONNX Model (Optional)

In [ ]:
# @title
!python -m onnxsim your_model.onnx simplified_model.onnx

#Convert ONNX to TensorFlow SavedModel

In [ ]:
# @title
#!pip uninstall -y tensorflow keras
#!pip install tensorflow==2.12 keras==2.12
#!pip install onnx onnx-tf
#!pip install tensorflow-probability==0.20.0

import onnx
import tensorflow as tf
from onnx_tf.backend import prepare

# Load the ONNX model
onnx_model = onnx.load("trained_model.onnx")

# Convert ONNX model to TensorFlow SavedModel
tf_rep = prepare(onnx_model)

# Cast all the weights and biases in the TensorFlow model to float32
for var in tf_rep.tf_module.variables:
    var.assign(tf.cast(var, tf.float32))

# Save the model in float32 precision
tf_rep.export_graph("/content/saved_model_dir_float32")

# Verify by loading the model and checking the dtype
saved_model = tf.saved_model.load("/content/saved_model_dir_float32")
print(saved_model.signatures)


#Verify Model Precision

In [ ]:
# @title
import tensorflow as tf

# Load the model
loaded_model = tf.saved_model.load('/content/saved_model_dir/')

# Inspect the model's signature to find the input tensor name
print(loaded_model.signatures)  # This will show the signature, including input tensor names

# Get the signature for serving
infer = loaded_model.signatures["serving_default"]

# Check the input signature name
print(infer.structured_input_signature)

# Define the correct input signature based on the inspected signature
@tf.function(input_signature=[tf.TensorSpec(shape=[None, 640, 640, 3], dtype=tf.float32)])
def predict(input_tensor):
    # Call the model using the correct input signature
    return infer(images=input_tensor)  # 'images' is typically the name of the input tensor for image models

# Add the signature when saving the model
signatures = {'serving_default': predict.get_concrete_function()}

# Save the model with the new signature
tf.saved_model.save(loaded_model, '/content/saved_model_dir_float32', signatures=signatures)


#Convert the TensorFlow SavedModel to TFLite

In [ ]:
# @title
import tensorflow as tf

# Load the TensorFlow SavedModel
converter = tf.lite.TFLiteConverter.from_saved_model('/content/saved_model_dir_float32/')

# Enable optimizations (optional)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Enable TensorFlow Select Ops
converter.allow_custom_ops = True

# Convert to TFLite format
tflite_model = converter.convert()

# Save the TFLite model
with open('tastetrace.tflite', 'wb') as f:
    f.write(tflite_model)

# Download the TFLite Model

In [ ]:
# @title
from google.colab import files
files.download('tastetrace.tflite')

# Conversion

In [ ]:
# @title

from ultralytics import YOLO

# Load the YOLO11 model
model = YOLO("/content/trained_model.pt")

# Export the model to TFLite format
model.export(format="tflite")  # creates 'yolo11n_float32.tflite'

# Load the exported TFLite model
tflite_model = YOLO("/content/trained_model_saved_model/trained_model_float32.tflite/")

# Run inference
results = tflite_model("https://ultralytics.com/images/bus.jpg")

#Fix the custom ops issue

In [ ]:
# @title
from ultralytics import YOLO
import tensorflow as tf
import onnx
from onnx_tf.backend import prepare

# **Step 1: Export YOLOv8 Model to ONNX**
model = YOLO('trained_model.pt')  # Replace with your YOLOv8 .pt model
model.export(format='onnx', opset=12)  # Adjust opset if necessary

# **Step 2: Convert ONNX to TensorFlow SavedModel**
onnx_model = onnx.load("trained_model.onnx")
tf_rep = prepare(onnx_model)

# Ensure all variables are in float32
for var in tf_rep.tf_module.variables:
    var.assign(tf.cast(var, tf.float32))

# Save the TensorFlow model in float32 format
tf_rep.export_graph("/content/saved_model_dir_float32")

# **Step 3: Replace Unsupported Ops in TensorFlow Model**
def replace_cast_op(saved_model_dir, output_dir):
    # Load the model
    loaded_model = tf.saved_model.load(saved_model_dir)

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 640, 640, 3], dtype=tf.float32)])
    def modified_model(input_tensor):
        # Replace unsupported `Cast` operations
        return loaded_model.signatures['serving_default'](input_tensor=tf.cast(input_tensor, tf.float32))

    # Save the modified model
    tf.saved_model.save(loaded_model, output_dir, signatures={'serving_default': modified_model.get_concrete_function()})

# Replace unsupported ops
replace_cast_op('/content/saved_model_dir_float32', '/content/saved_model_dir_fixed')

# **Step 4: Convert TensorFlow Model to TensorFlow Lite**
converter = tf.lite.TFLiteConverter.from_saved_model('/content/saved_model_dir_fixed')

# Enable optimization (optional)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Enable TensorFlow Select Ops to resolve `Cast` issue
converter.allow_custom_ops = True

# Convert the model
tflite_model = converter.convert()

# Save the TFLite model
with open('tastetrace.tflite', 'wb') as f:
    f.write(tflite_model)

# **Step 5: Verify the Converted TFLite Model**
interpreter = tf.lite.Interpreter(model_path='tastetrace_fixed.tflite')
interpreter.allocate_tensors()
print("TFLite model loaded successfully!")


In [ ]:
# @title
!pip install tensorflow-addons==0.20.0
!pip install tensorflow==2.12